In [111]:
import pandas as pd
import time
import string
from functions.capitalization import titlecase
so_file='fwso/sample txtfile.txt'
widths = [4,2,15,10,30,100,4,30,3,7,1,30,30,2,20,2,2,10,9,50,3,30,50]
update=pd.read_fwf(so_file,
                   header=None, 
                   widths=widths,
                   names=['dir_code','so_type','sonumber','mem_wstd','first_name','last_name','sam_city'
                          ,'sam_stname','sam_stsubt','sam_stnmfr','sam_subd','sam_estate','sam_bldname','sam_aptcod','sam_aprmnt'
                          ,'list_code','acc_type','old_wstd','so_date','joint_user','regulatory_id','account_no','distribution_code'],
                   converters = {'mem_wstd': str, 'sam_stnmfr':str,'account_no':str},
                   index_col=None
                  )
#update.()
update = update.fillna('')

update.sort_values(by='last_name',inplace=True)
update.reset_index(drop=True,inplace=True)
update['acc_type']=update['acc_type'].astype('category')
update['src']=so_file
update['so_rangedate']=time.strftime("%Y/%m/%d")
rr=update

In [112]:
rr_crm=pd.DataFrame()

In [113]:
rr_crm['Areacode']=rr.mem_wstd.str.slice(0,1)
rr_crm['Phone']=rr.mem_wstd.str.slice(-7)
rr_crm['name1']=rr.last_name
rr_crm['name2']=rr.first_name
rr_crm['SAM_BLDNAME']=rr.sam_bldname
rr_crm['SAM_STNMFR']=rr.sam_stnmfr
rr_crm['SAM_STNAME']=rr.sam_stname
rr_crm['SAM_STSUBT']=rr.sam_stsubt
rr_crm['sam_estate']=rr.sam_estate
rr_crm['City']=rr.distribution_code.str.split('  ',1).str.get(1).apply(lambda x : x.strip())
rr_crm['Province']=rr.distribution_code.str.split('  ',1).str.get(0).apply(lambda x : x.strip())
rr_crm['class_code']=""
rr_crm['class_desc']=""
rr_crm.name1 = rr_crm.name1.apply(lambda x: titlecase(x))
rr_crm.name2 = rr_crm.name2.apply(lambda x: titlecase(x))
rr_crm.SAM_BLDNAME = rr_crm.SAM_BLDNAME.apply(lambda x: titlecase(x))
rr_crm.SAM_STNAME = rr_crm.SAM_STNAME.apply(lambda x: titlecase(x))
rr_crm.SAM_STSUBT = rr_crm.SAM_STSUBT.apply(lambda x: titlecase(x))
rr_crm.sam_estate = rr_crm.sam_estate.apply(lambda x: titlecase(x))
rr_crm.City = rr_crm.City.apply(lambda x: titlecase(x))
rr_crm.Province = rr_crm.Province.apply(lambda x: titlecase(x))

In [114]:
import numpy as np
def or_call(crm):
    index=crm.duplicated(['name1','name2','City','Province','SAM_BLDNAME','SAM_STNAME'])
    skip=False
    for i in xrange(len(index)):
        if index.ix[i] == True:
            line=crm.ix[i]
            prev_line=crm.ix[i-1]
            try:
                next_line=crm.ix[i+1]
            except:
                pass
            line['SAM_BLDNAME']=''
            line['SAM_STNMFR']=''
            line['SAM_STNAME']=''
            line['SAM_STSUBT']=''
            line['sam_estate']=''
            line['Province']=''
            line['class_code']=''
            line['class_desc']=''
            #line['name1']=''
            #line['name2']=''
            if skip == False:
                prev_line['Phone']=str(prev_line.Phone)+'/'+str(line.Phone)
                line['Phone']=''

                if index.ix[i-1] == False and index.ix[i+1]==True:
                    next_line['City']='Or Call:'
                try:
                    if index.ix[i+1] == True:
                        skip =True
                    
                except:
                    pass
            else:
                skip = False
    crm['Phone'].replace('', np.nan, inplace=True)
    return  crm[crm.Phone.notnull()]

    

In [115]:
import csv 
with open('areacodes') as csvfile:
    reader= csv.reader(csvfile)
    codes = dict((rows[0],rows[1]) for rows in reader)


IOError: [Errno 2] No such file or directory: 'areacodes'

In [116]:
codes.get('bulacan')

'44'

In [117]:
rr_crm.duplicated('name1')

0     False
1     False
2     False
3      True
4     False
5     False
6      True
7      True
8     False
9     False
10    False
11    False
12    False
13     True
14     True
dtype: bool

In [126]:
or_call(rr_crm).City[0]

'Obando'

In [119]:
str(open('small_words').read())

IOError: [Errno 2] No such file or directory: 'small_words'

In [120]:
open('small_words').read().rstrip('\n')


IOError: [Errno 2] No such file or directory: 'small_words'

In [121]:
import functions.interface_functions

In [122]:
import glob
import os
newest = min(glob.iglob('database_backups/*.hdf5'), key=os.path.getctime)


In [62]:
newest

'database_backups/database-2016-07-21-02-02-20.hdf5'

In [63]:
rr.distribution_code.str.split('    ',1).str.get(1)

0                                 OBANDO
1                        MEYCAUAYAN CITY
2                                 OBANDO
3                                 OBANDO
4                        MEYCAUAYAN CITY
5                                 OBANDO
6                                 OBANDO
7                                 OBANDO
8                                 OBANDO
9                                 OBANDO
10                                OBANDO
11                                OBANDO
12                                OBANDO
13                                OBANDO
14                                OBANDO
Name: distribution_code, dtype: object

In [66]:
index=rr_crm.duplicated(['name1','name2','City','Province','SAM_BLDNAME','SAM_STNAME'])


In [69]:
range(len(index))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [74]:
index[1]

False

In [72]:
index=rr_crm.duplicated(['name1','name2','City','Province','SAM_BLDNAME','SAM_STNAME'])


In [80]:
'dfdsf sdfsdfs        sdfsd'.split('s')

['dfd', 'f ', 'df', 'df', '        ', 'df', 'd']